# CHRL Wx Station Pretrip Report
Script for use with '**Weather Station Visit Form**' GoogleSheet (== googlesheet automatically updated by device magic)

In [52]:
import gspread
import pandas as pd
import numpy as np
from datetime import date

**USER INPUTS:**

In [ ]:
# Select station
station = 'Steph 3'
# Select number of recent entries to include
num_entries = 5

In [53]:
# provide credentials and acccess googlesheet
gc = gspread.service_account(filename=r'C:\Users\OCONNORB\AppData\Local\gspread\python-pull-wxstation-dmforms-1c0dcc60196b.json')
sh = gc.open("Weather Station Visit Form")

# pull wx station visit sheet and put in dataframe
worksheet = sh.sheet1;
df = pd.DataFrame(worksheet.get_all_records())

# drop duplicates created by snow course (= multiple entries per visit) *unless we want snow course info?
df.drop_duplicates(subset=['submissionid'], inplace=True, ignore_index=True)

# get entries for target station
df = df.loc[np.where(np.any(df == station, axis=1))]

# select columns to keep in table and define new column names for each
cols2keep = ['Job_Start_Time', 
             'User',
             'What_jobs_are_being_completed_.Snow_Course',
             'What_jobs_are_being_completed_.Drone_Survey',
             'What_jobs_are_being_completed_.CF',
             'What_jobs_are_being_completed_.Sensor_Change',
             'What_jobs_are_being_completed_.Precip_Gage',
             'What_jobs_are_being_completed_.Lys_Calibration',
             'What_jobs_are_being_completed_.Tipping_Bucket_Calibration',
             'What_jobs_are_being_completed_.Data_Download',
             'What_jobs_are_being_completed_.General_Maintenance',
             'Sensor_Change.Type_of_Sensor',
             'Sensor_Change.Why_is_the_sensor_being_changed',
             'Sensor_Change.Additional_Notes',
             'General_Notes']
new_colnames = ['date', 'users', 'snow_course', 'drone', 'CF', 'sens_change', 'p_gage', 'lys_cal', 'buck_cal', 'data', 'gen_maint', 'sens_changed','reason', 'sens_notes', 'general_notes']

# get selected columns
df = df[cols2keep].set_axis(new_colnames, axis='columns')

# find the n most recent entries
df = df.sort_values(by="date")
df_table = df.iloc[-5:, :].copy()

# clean up for table
df_table[df_table == 'no'] = ' '
df_table[df_table == 'yes'] = 'Y'

# convert to html file
filestr = station.replace(" ", "").replace("-", "") + '_pretrip_' + date.today().strftime("%d %b %Y").replace(" ", "") + '.html'
df_table.to_html(open(filestr, 'w', encoding="utf-8"), index=False)

# some code for pdf conversion (missing config definition above)
# with open(station.replace(" ", "") + '_pretrip_example.html') as f:
#     pdfkit.from_file(f, station.replace(" ", "") + '_pretrip_example.pdf',configuration=config)